# LASIF with Salvus

LASIF can in theory be used in combination with any wave propagation solver which solves both the forward and the adjoint wave equation. The solver which integrates most easily with [LASIF](https://dirkphilip.github.io/LASIF_2.0/) is [Salvus](https://mondaic.com) as this version of LASIF was originally developed for Salvus and currently contains some nice utilities which ease the combination of the two.

In this tutorial we will show how we can combine the two and create a nice workflow. This tutorial does not contain any sort of optimization routines to progress through iterations but only general things of how to perform forward and adjoint simulations and the steps that are needed in between. _LASIF_ works both through command line and an API. In this tutorial we will use the API as we believe it is a nicer interface and it can be used for scripting.

The _Salvus_ support of _LASIF_ is currently based on _Salvus_ version 0.11.0 or higher. If you have an older version, the only thing you need is to go into the `salvus_utils.py` file inside _LASIF_ and change all the calls to _Salvus_ python packages from `salvus.subpackage` to `salvus_subpackage`.

For _LASIF_ installation, we refer to these [installation instructions](https://dirkphilip.github.io/LASIF_2.0/installation.html#installation).

Basically all LASIF's state information is persistent and one should not assume any in-memory data really matters. So if you return to this notebook and want to continue where you ended up last time, you should skip everything except the needed imports right below. If you see anything that is not persistent, it should be considered a bug, so please let us know.

__Author of tutorial:__
_Solvi Thrastarson_
soelvi.thrastarson@erdw.ethz.ch


In [ ]:
# Input of various libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import pathlib
import pprint
from typing import Union, Dict, List

In [ ]:
import obspy
import lasif.api
from lasif import salvus_utils

In [ ]:
from salvus.mesh.simple_mesh import basic_mesh

## Define a project

Here we define our project and make the initial setup required to make a project run.

In [ ]:
# Project Definition
project_name = "lasif_project"
lasif_root = pathlib.Path(os.getcwd()) / project_name

In [ ]:
# Initialize the project if it not yet exists
if not lasif_root.exists():
    lasif.api.init_project(project_path=lasif_root)

The directory structure which _LASIF_ has created can be visualized using the `tree` command. (You may have to install the software to do so)

In [ ]:
!tree

The directory structure is designed to incorporate all files you might accumulate during an inversion in an easily understandable way.
There is an experimental support for correlations (not guaranteed to work).
One key file that we can see is the `lasif_config.toml`.
This is where you define all the relevant parameters for your inversion.
It is a toml file which can be loaded in an operated on as a Python dictionary.

In [ ]:
# Read default config file
config_file_path = lasif_root / "lasif_config.toml"
config_dict = toml.load(config_file_path)

In [ ]:
# Read input file with user-specified configuration that will override default settings
# The input file can contain any subset of keys of lasif_config.toml with custom values
input_file_path = pathlib.Path(os.getcwd()) / "lasif_input.toml"
input_dict = []
if input_file_path.exists():
    input_dict = toml.load(input_file_path)

In [ ]:
# Print user-specified configuration
pprint.PrettyPrinter().pprint(input_dict)

In [ ]:
# Merge user-specified configuration into the complete configuration
if input_dict:
    try:
        import mergedeep
        mergedeep.merge(config_dict, input_dict)
    except ImportError as e:
        print("Warning: mergedeep package not installed, cannot merge in input file")
        pass

In [ ]:
# You can also change settings programatically
# Change download length
download_after = 1200.0
config_dict["lasif_project"]["download_settings"]["seconds_after_event"] = download_after

In [ ]:
# Print complete configuration
pprint.PrettyPrinter().pprint(config_dict)

In [ ]:
with open(config_file_path, "w") as fh:
    toml.dump(config_dict, fh)

As you can see there are many things to configure there. Usually they are explained through comments but if anything is unclear, feel free to ask. Through the `toml` library it is possible to edit this config at will as long as the correct keys are used in the dictionary and the correct types are inserted as values.

## Define the domain

Since this is a _LASIF_ with _Salvus_ tutorial we can use _Salvus_ to create a mesh which defines our domain. We will thus use one of the easy-to-use built-in _Salvus_ meshes, courtesy of Martin van Driel.

In [ ]:
# Create mesh
def create_spherical_chunk(file_location: Union[pathlib.Path, str], mesh_info: dict):
    """
    Create a mesh and write it to file.
    
    :params file_location: Path to the where the mesh should be stored.
    :type file_location: Union[pathlib.Path, str]
    :params mesh_info: basic_info needed for the mesh construction
    :type mesh_info: Dict[str, float]
    """
    
    m = basic_mesh.SphericalChunkBuiltIn3D(
        model=mesh_info["model"],
        period=mesh_info["period"],
        latitude_extent=mesh_info["latitude_extent"],
        longitude_extent=mesh_info["longitude_extent"],
        tensor_order=mesh_info["tensor_order"],
        latitude_center=mesh_info["latitude_center"],
        longitude_center=mesh_info["longitude_center"],
        elements_per_wavelength=mesh_info["elements_per_wavelength"],
        minimum_radius_in_km=mesh_info["minimum_radius_in_km"]
    )
    mesh = m.create_mesh()
    mesh.write_h5(file_location)
    
    return mesh

In [ ]:
# We can write the mesh into the location where the lasif_config.toml already thinks it is
file_location = config_dict["lasif_project"]["domain_settings"]["domain_file"]
mesh_info = {
    "model": "prem_ani_one_crust",
    "period": 30.0,
    "latitude_extent": 9.0,
    "longitude_extent": 11.0,
    "tensor_order": 4,
    "latitude_center": 41.0,
    "longitude_center": 12.5,
    "elements_per_wavelength": 1.5,
    "minimum_radius_in_km": 5200.0,
}

In [ ]:
mesh = create_spherical_chunk(file_location=file_location, mesh_info=mesh_info)
mesh

So now _LASIF_ should know it's domain and we defined in the `lasif_config.toml` a specific boundary length which we also have to consider. This boundary length `lasif_project.domain_settings.boundary_in_km` is only used when deciding whether datapoints are inside the domain or not. We will leave this parameter at the initial value and continue. 

This parameter is not to be confused with the absorbing boundaries. There is another parameter for actual absorbing boundaries in the `simulation_settings` of the config.

## Finding Sources

_LASIF_ has two ways of finding sources. One via the [GCMT catalog](https://www.globalcmt.org/CMTsearch.html) and another one via [IRIS spud events](http://ds.iris.edu/spud/momenttensor). The GCMT way is nice when you just want to get some events with a certain criteria which fit into your domain. The spud events work better if there are specific events that you want to use.

In this tutorial we'll use the GCMT catalog. When looking for events it filters the catalog to only have the events which fit the specified criteria and uses the [Mitchell's best candidate](https://bl.ocks.org/mbostock/d7bf3bd67d00ed79695b) algorithm to select events with a somewhat optimal distribution.

In [ ]:
# Find earthquakes
lasif.api.add_gcmt_events(
    lasif_root=lasif_root,
    count=2,
    min_mag=4.5,
    max_mag=5.5,
    min_dist=100.0,
    min_year=2012
)

As requested, Lasif found 2 earthquakes for us. We can now plot these events to see their distribution.

In [ ]:
# Plot events
lasif.api.plot_events(
    lasif_root=lasif_root
)

## Download Data

_LASIF_ uses the [Obspy](https://github.com/obspy/obspy/wiki) MassDownloader to download the available data for the sources in the project. It is possible to configure this downloader to do many different things. Some things you can modify in the LASIF config file but in many cases I would recommend just looking into the downloader in lasif and modifying it at will in there if you have any special requests.

In [ ]:
# Download data
lasif.api.download_data(
    lasif_root=lasif_root,
)

It can take a while to download data, but one way to accelerate it is to specify which provider you want to get the data from and to configure the settings of the mass downloader.

But since the data is ready, we can visualize the distribution of stations and such.

In [ ]:
# Plot rays for one event
event = lasif.api.list_events(lasif_root, output=True)[0]
lasif.api.plot_event(
    lasif_root,
    event_name=event,
)

We can plot the raydensity of our project where the rays are coloured based on how many other rays each ray crosses on the way.

In [ ]:
# Plot raydensity
lasif.api.plot_raydensity(
    lasif_root,
    plot_stations=True,
    save=False,
)

Alternatively we can simply plot all the rays in the project, without event considering the crossings. In this plot, rays from each event get the same color. The colors are currently completely random so this plot always has the potential to turn out terribly ugly but sometimes life is like that.

In [ ]:
# Plot all rays:
lasif.api.plot_all_rays(
    lasif_root,
    plot_stations=True,
    save=False,
)

## Set up an iteration

_LASIF_ organizes things through iterations. Each folder, except data and processed data, will have a subfolder with iterations where everything related to each iteration is stored. This creates the flexibility of using subsets of events per iteration. You can thus always define iterations on the full dataset or subsets of it and whenever you pass an iteration argument to one of the _LASIF_ functions, it will know what applies to the relevant iteration.

Now, however we will use all of our events as we only have two events in our dataset.

In [ ]:
# Set up iteration
first_iteration = "first"
lasif.api.set_up_iteration(lasif_root, iteration=first_iteration)

All the plotting functions take iterations as arguments, so we can plot only the events defined in each iteration. In this case it doesn't make a difference but it would be done this way:

In [ ]:
# Plot events for first iteration
lasif.api.plot_events(
    lasif_root,
    iteration=first_iteration
)

We can now see how the folder structure has changed by printing the folder tree again. We should both see some folders related to the new iteration and some data file since we found the events.

In [ ]:
!tree

## Run simulations

In order to see how good our starting model is we have to run wave propagation simulations.

Before running such simulations it is good to look into what is important to keep in mind when running simulations on a computational cluster.

* Find a timestep: You can initialize a short simulation on Salvus with the mesh you plan to use and take a look at the timestep that Salvus proposes. You should choose something very close to that timestep (preferrably shorter) which is a good number that you are comfortable with and you want your endtime to be a multiple of that number.

* Number of cores to use: It is a good rule of thumb to aim for about ~5000 elements per core and try not to go below ~1000 elements per core. This guarentees good scaling with cores, otherwise communication becomes dominant and scaling will plateau.

* Wall time: Keep in mind that wall time is only used to place you in the queue and it is not the time you will in the end be billed for. So make sure that the wall time you set is long enough to finish your simulation as the simulation will be killed if the wall time is passed. Also keep in mind that when using checkpointing, your adjoint simulation is about 3 times more expensive than your forward one.

* Cluster to use: This you have to set up yourself, using [SalvusFlow](https://mondaic.com/docs/installation/salvus_flow_configuration) where you can find some information on how to set things up correctly.

A part of the _LASIF_ library is a file called _salvus_utils.py_ where there are some functions which ease the usage of _Salvus_ with _LASIF_. We will demonstrate some of those now. But make sure that you already configured your cluster settings and update your __lasif_config.toml__ file accordingly.

In [ ]:
# Update toml file
wall_time_in_s = 3600
# Our mesh has about 16.600 elements so we don't want more than 16 ranks.
# One node on Daint is 12 ranks so we'll use that although it is on the higher end for optimal scaling.
ranks = 12
site_name = "daint"
min_period = 35.0
# We recommend keeping the salvus absorbing boundaries a bit smaller than the domain ones
# as it is important to make sure we do not download receivers within the absorbing boundaries
absorbing_boundaries_in_km = 50.0
gradient_parameterization = "tti"
config_dict["salvus_settings"]["wall_time_in_s"] = wall_time_in_s
config_dict["salvus_settings"]["ranks"] = ranks
config_dict["salvus_settings"]["site_name"] = site_name
config_dict["salvus_settings"]["absorbing_boundaries_in_km"] = absorbing_boundaries_in_km
config_dict["salvus_settings"]["gradient_parameterization"] = gradient_parameterization
config_dict["simulation_settings"]["minimum_period_in_s"] = min_period


with open(config_file_path, "w") as fh:
    toml.dump(config_dict, fh)

In [ ]:
# Create forward simulation objects

# We need the lasif communicator object to use the salvus utils:
lasif_comm = lasif.api.find_project_comm(lasif_root)
events = lasif.api.list_events(lasif_root, just_list=True, iteration=first_iteration, output=True)
simulations = []
for event in events:
    simulation = salvus_utils.create_salvus_forward_simulation(
        comm=lasif_comm,
        event=event,
        iteration=first_iteration,
        side_set="r1"
    )
    simulations.append(simulation)

In a Jupyter notebook the simulations can even be plotted which is nice to make sure everything looks correct.

In [ ]:
# Plot simulations
simulations[0]

Now that we have created the Salvus simulation objects we can use _SalvusFlow_ to submit these simulations. The nice thing about _SalvusFlow_ is that it abstracts the cluster away from the workflow meaning that you do the exact same thing on your side whether you are working on your local computer or any other computer.

When we submit our simulations we have two options. We can submit them one by one and have separate job objects for each of them or we can submit them as a _JobArray_ and then they have a group JobArray object as well as their own ids. _LASIF_ keeps track of the job names and can figure it out for you how to query their statuses.

Whether a _JobArray_ or a _Job_ will be submitted depends on your input into the function. If you input a list of simulations, an array will be submitted but if you send one simulation, a single job will be submitted. Make sure that you submit a list of events which correspond to the list of your simulation objects. _LASIF_ can handle both the arrays and the single jobs.

In [ ]:
# Run forward simulations
salvus_utils.submit_salvus_simulation(
    comm=lasif_comm,
    simulations=simulations,
    events=events,
    iteration=first_iteration,
    sim_type="forward"
)

## Process data

While we wait for the simulation to be done, it is a good time to perform some tasks that we need to do anyway, like processing our raw data. Processing data can be done on an event level, an iteration level or full dataset level, it all depends on the input. The default is the full dataset, which we will do now.

Don't worry although it throws some errors. The script will process what data it can actually process. Unfortunately it seems to be unavoidable that when downloading data, a large chunk of the data we get are quite flawed and doesn't have all the relevant information we require. These data will be discarded.

In [ ]:
# Process data

lasif.api.process_data(lasif_root)

## Station weighting

In an inversion some regions with a high density of stations can become quite dominant in the resulting gradients as they emit more energy into the adjoint simulations compared to underrepresented regions. Partly this is a natural things which makes sense as more data means more information and should thus to some extent dominate an inversion but on the other hand this can also slow down convergence.

In _LASIF_ there is a function which computes weights for stations based on their geographical distribution. Using these weights reduces the effect of oversampled regions and we thus recommend using them as you only need to compute these weights once, it is very fast, and can potentially speed up your inversion.

In [ ]:
# Compute station weights
station_weights = "stations"
lasif.api.compute_station_weights(
    lasif_root,
    weight_set=station_weights,
    iteration=first_iteration
)

We can plot the station weights using the _plot_event_ function by passing a weight set argument to it. This is not the most beautiful plot currently. But I'll fix it, no worries.

In [ ]:
# Plot station weights:
event = lasif.api.list_events(
    lasif_root,
    output=True,
)[0]
lasif.api.plot_event(
    lasif_root,
    event_name=event,
    weight_set_name=station_weights,
)

## Retrieve simulation outputs

_SalvusFlow_ defines it's own location where it runs its simulations and after they are done, we need to download the relevant output into the correct locations for _LASIF_ to recognize them. The Salvus utils can do this for you based on the job names it saved before.

When you ask _LASIF_ to retrieve outputs from a simulation, it will check whether the simulation terminated successfully, terminated with an error or whatever else might be going. It can obviously only download the output if the simulation terminated successfully, otherwise it will report the status of the simulation.

In [ ]:
# Retrieve synthetic seismograms
lasif_comm = lasif.api.find_project_comm(lasif_root)
events = lasif.api.list_events(lasif_root, just_list=True, iteration=first_iteration, output=True)
lasif.salvus_utils.retrieve_salvus_simulations(
    comm=lasif_comm,
    events=events,
    iteration=first_iteration,
    sim_type="forward"
)

## Graphical user interface

_LASIF_ is equipped with a Graphical User Interface (GUI) which can be used to plot waveforms from specific stations, explore windows and compare waveforms between iterations. The GUI can either be opened from the command line by typing `lasif gui` into a terminal window or via the API by running the cell below. I would not really recommend it though as it tends to kill the kernel. Rather just open it from the terminal.

In [ ]:
# Open GUI
lasif.api.open_gui(lasif_root)

## Window selection

When comparing waveforms it is important that they are sufficiently similar in order to do a meaningful measurement of the difference between them. The main things to keep in mind are:

* Phase jumps: When two waves are compared they are often compared in terms of phase and then the wiggle closest to the reference wiggle is considered to be the relevant wiggle of comparison. But if the phase shift between the wiggles is too large, the correct wiggle might not be the closest one anymore resulting in a completely wrong measurement.

* Noise: If a trace has a high noise level then it is hard to separate the signal from the noise. A signal to noise ratio test is thus of great importance and we do not want to use traces with a low ratio. Quality of signal over quantity in that sense.

The window selection algorithm in _LASIF_ tries to account for these two things and selects parts of the waveforms which can be compared in a meaningful way. As your model gets better, larger parts of the waveforms become measurable and it can thus be of great benefit to be able to repick windows as often as possible. How feasible it is to repick windows regularly depends on your inversion algorithm.

It is impossible to be completely correct with such window selection algorithms and the same can be said about human picking. With the vast data volumes we have today it is a huge waste of time to manually pick all windows in your waveforms and thus the best thing we can do is to try to improve our window picking algorithms.

What the window selection does to your inversion is essentially to linearize it. Through possible phase jumps and elevated noise levels many local minima can appear in your misfit function and we want to get past this. That is the reason we pick windows.

_LASIF_ can pick windows in parallel if you call it from the command line, but it can also pick windows via the API, just not parallelly currently. Since we have a small dataset here we will use the API.

In [ ]:
# Select windows
window_set="A"
lasif.api.select_windows_multiprocessing(lasif_root, iteration=first_iteration, window_set=window_set)

## Compute adjoint sources

As we have estimated where we can make meaningful measurements between synthetics and data, it is time to do these measurements. In _LASIF_ there are a few different misfit measurements which can be used. Most importantly the classic L2 waveform misfit:

$$ \chi = \frac{1}{2} \int_{t=0}^{t=\tau}(d_{syn}(t) - d_{obs}(t))^2 dt $$

That one is probably the most straightforward and always works best in any synthetic test case where our physics are perfectly modelled. The L2 misfits takes in amplitude information which we do not understand as well as we do phase information and it can thus be a bit dangerous in practice. There are some remedies to that like using normalized L2, regularized L2 or starting with a different misfit function and do a few L2 iterations at the end when you are sort of sure you are in the vicinity of a good minimum. L2 is usually far from convex and is thus a bit dangerous in real data cases.

Another one is the cross correlation time shift where we keep one trace fixed and time shift the other one within a certain time range:

$$ \chi = argmax((f(t) * g(t-\tau))$$

The $\tau$ which produces the highest correlation between the two signals is picked as the time shift between the two. Problem with this misfit function is that as opposed to the L2 which has information on every time step of the waveform, the cross correlation simply makes one measurement for each window in the seismogram.

Then there is the misfit which we will use in this tutorial, the time-frequency phase misfit which we will not describe here but refer to [Fichtner et al. 2008](https://onlinelibrary.wiley.com/doi/10.1111/j.1365-246X.2008.03923.x). Maybe at some point more info on that will be added here.

The same parallel relation applies to the adjoint source calculations as did with the window selection.

In [ ]:
# Compute adjoint sources
lasif.api.calculate_adjoint_sources_multiprocessing(
    lasif_root,
    iteration=first_iteration,
    window_set=window_set,
    weight_set=station_weights
)

## Plot misfits

Now that we have computed misfits, we can plot how the misfit is distributed between stations. This can help you realize whether there is something fishy going on or not. This plot is probably the most meaningful with a cross-correlation misfit.

In [ ]:
# Plot station misfits
event = lasif.api.list_events(
    lasif_root,
    output=True,
)[0]
lasif.api.plot_station_misfits(
    lasif_root=lasif_root,
    event=event,
    iteration=first_iteration
)

## Create adjoint simulation

In the same way as we created a Salvus simulation object before, we have to do exactly the same thing for the adjoint simulations.

We do this in basically the exact same way as the forward simulation, except now we change the _sim_type_ attribute to "adjoint". Simple as that.

In [ ]:
# Create adjoint simulations
lasif_comm = lasif.api.find_project_comm(lasif_root)
events = lasif.api.list_events(lasif_root, just_list=True, iteration=first_iteration, output=True)
simulations = []
for event in events:
    simulation = salvus_utils.create_salvus_adjoint_simulation(
        comm=lasif_comm,
        event=event,
        iteration=first_iteration,
    )
    simulations.append(simulation)

## Submit adjoint simulation

This is done in the exact same way as the forward simulation, the only need is to change the _sim_type_ attribute to "adjoint". It's as simple as that. Keep in mind though that the adjoint simulation is about 2-3 times more expensive than the forward one meaning that you might need an increased wall time.

In [ ]:
# Submit adjoint simulation
lasif.salvus_utils.submit_salvus_simulation(
    comm=lasif_comm,
    simulations=simulations,
    events=events,
    iteration=first_iteration,
    sim_type="adjoint",
)

## Retrieve gradients

Again this works just the same as the retrieving of the seismograms. It checks the job status and acts accordingly. It can sometimes take a bit of time to figure out the status of each job. In order to reduce the time, it is good to regularly empty or clean your _SalvusFlow_ database. Let's try to retrieve our jobs.

In [ ]:
# Retrieve gradients
lasif_comm = lasif.api.find_project_comm(lasif_root)
events = lasif.api.list_events(lasif_root, just_list=True, iteration=first_iteration, output=True)
lasif.salvus_utils.retrieve_salvus_simulations(
    comm=lasif_comm,
    events=events,
    iteration=first_iteration,
    sim_type="adjoint",
)

As you open the gradient files in [Paraview](https://www.paraview.org/) you'll notice that the sensitivities are mostly focussed around the sources and receivers. For that reason, sometimes small regions around the sources and the receivers are cut away from the gradients and the gradient then smoothed. This effect is strongest around the source, so I would especially recommend cutting that region out.

# End of tutorial

Congratulations, you've made it to the end of this tutorial.

What has been described for you is what _LASIF_ can do for you in a full-waveform inversion using _Salvus_. There are obviously things that _LASIF_ does not do for you. Namely

* Regularization - You can either implement this yourself or use what is implemented in Salvus
* Optimization - As FWI is a non-linear optimization problem, you need to figure out what to do between iterations. This is, like the regularization, something you can either implement your self, or use what is implemented in Salvus.

There are currently no plans to include this in _LASIF_. If you are interested in running your inversion automatically, you might want to take a look at [_Inversionson_](http://www.github.com/solvithrastar/inversionson) which uses _LASIF_ and _SALVUS_ and fully automates the inversion workflow.

## Current future LASIF plans

As time becomes available we have some things that we would like to improve in _LASIF_. One of those things is to add some new plotting functionalities which make it easier to follow the process of the inversion. This will hopefully be added in the very near future.

There have also been some discussion of making _LASIF_ support other solvers in a way it supports _Salvus_. As _LASIF_ is independent of _Salvus_, it can be used with other solvers but further support might be helpfull.

### Suggestions

If you have any ideas of what might be a nice addition to _LASIF_ feel free to either send us a suggestion, via email (soelvi.thrastarson@erdw.ethz.ch) or on [Github](www.github.com/dirkphilip/LASIF_2.0), or to code it up yourself and submit a pull request.